# Feature Engineering
Feature Engineering is the key.

  
reference:  
https://www.kaggle.com/mjbahmani/statistical-analysis-for-elo  
https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/82055  
https://www.kaggle.com/chauhuynh/my-first-kernel-3-699  
https://www.kaggle.com/fabiendaniel/elo-world  
https://www.kaggle.com/raddar/target-true-meaning-revealed  
https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/82036#479038  
https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/  
https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering this is really helpfull

In [39]:
import pandas as pd
import os
import numpy as np
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
# import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

print(os.listdir("../data"))

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', 'merchants.csv', 'historical_transactions.csv', 'train.csv', 'load_data.py', 'sample_submission.csv']


### Load Data

In [40]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
merchants = pd.read_csv('../data/merchants.csv')
historical_transactions = pd.read_csv('../data/historical_transactions.csv')
new_transactions = pd.read_csv('../data/new_merchant_transactions.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')
print('...loaded')

### Fill Null Data

In [42]:
# view NA data
print("train set NA data {}".format(train.isna().sum()))
print("test set NA data {}".format(test.isna().sum()))
print("new transaction set NA data {}".format(new_transactions.isna().sum()))
print("historical transaction set NA data {}".format(historical_transactions.isna().sum()))

train set NA data first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64
test set NA data first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64
new transaction set NA data authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64
historical transaction set NA data authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
categ

In [43]:
# drop NA data, be careful
train = train.dropna()

# fill the NA data, is it going to have a bad influence on our model?
for df in [historical_transactions, new_transactions]:
    df['category_3'].fillna('A', inplace=True)
    df['category_2'].fillna(1.0, inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)

### Feature Extraction

Manual feature engineering can be a tedious process (which is why we use automated feature engineering with featuretools!) and often relies on domain expertise. I will concentrate of getting as much info as possible into the final training dataframe. The idea is that the model will then pick up on which features are important rather than us having to decide that. Basically, our approach is to make as many features as possible and then give them all to the model to use! Later, we can perform feature reduction using the feature importances from the model or other techniques such as PCA.

In [44]:
# print all current features
print(list(train))
print(list(test))
print(list(historical_transactions))
print(list(new_transactions))

['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target']
['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3']
['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3', 'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2', 'state_id', 'subsector_id']
['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3', 'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2', 'state_id', 'subsector_id']


We need to understand the feature now.
1. the train dataset, we can see that it has card_id, this should be the unique id we can use for user profile, and group or aggregate the data w.r.t this feature. the feature_# in the data_dictionary descibed as Anonymized card categorical feature, we need to understand it later.
2. historical and new transactions. these two dataset have similar data structure.  
    a. 'purchase_date': max','min', require feature extraction.
        - 'month','hour','weekofyear','dayofweek','year', require no further process
        - 'weekend': 'sum', 'mean'
    b. 'merchant_id', 'merchant_category_id', 'subsector_id' need no more process.  
    c. 'purchase_amount': 'sum','max','min','mean','var'.  
    d. 'month_lag': 'max','min','mean','var'  
    e. 'month_diff': 'mean'  
    f. 'authorized_flag': 'sum, 'mean'  
    g. 'category_1': 'sum', 'mean'  
    h. 'category_2': 'mean'  
    i. 'category_3': 'mean'  
    j. 'card_id': 'size'

In [45]:
# helper function, create new column
def create_column(prefix, aggs):
    return [prefix + '_' + key + '_' + agg for key in aggs.keys() for agg in aggs[key]]

In [46]:
## main data preprocessing block

# 1. One important feature here is purchase_date feature, we need to extract it into year, month,
# week of year, day of week, weekend, hour
# 2. get time difference to today which 
# 3. normalize binary data to 1/0 int


for df in [historical_transactions, new_transactions]:
    
    # date conversion
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df['purchase_date'].dt.dayofweek >= 5).astype(int) # 0-5 week day
    df['hour'] = df['purchase_date'].dt.hour
    
    ## time difference
    # https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
    # normalization
    # TODO still not well done here
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 1, 'N': 0})
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0})
    


In [24]:
df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,year,weekofyear,month,dayofweek,weekend,hour,month_diff
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1,-0.557574,...,1.0,9,19,2018,10,3,6,1,14,14
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1,-0.569580,...,1.0,9,19,2018,12,3,0,0,18,13
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2,-0.551037,...,1.0,9,14,2018,17,4,3,0,14,13
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1,-0.671925,...,1.0,-1,8,2018,10,3,2,0,9,14
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1,-0.659904,...,1.0,-1,29,2018,12,3,3,0,21,13


### Feature aggregation
`groupby`: group a dataframe by a column. In this case we will group by the `card_id`.  
`agg`: perform a calculation on the grouped data such as taking the `mean` of columns. We can either call the function directly (grouped_df.mean()) or use the agg function together with a list of transforms (`grouped_df.agg([mean, max, min, sum])`)  
`merge`: match the aggregated statistics to the appropriate client. We need to merge the original training data with the calculated stats on the `card_id` column.

In [47]:
# create aggregation columns (features)
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['card_id'] = ['size']

aggs['category_1'] = ['sum', 'mean']

for col in ['category_2','category_3']:
    aggs[col+'_mean'] = ['mean'] 

In [48]:
aggs

{'month': ['nunique'],
 'hour': ['nunique'],
 'weekofyear': ['nunique'],
 'dayofweek': ['nunique'],
 'year': ['nunique'],
 'subsector_id': ['nunique'],
 'merchant_id': ['nunique'],
 'merchant_category_id': ['nunique'],
 'purchase_amount': ['sum', 'max', 'min', 'mean', 'var'],
 'installments': ['sum', 'max', 'min', 'mean', 'var'],
 'purchase_date': ['max', 'min'],
 'month_lag': ['max', 'min', 'mean', 'var'],
 'month_diff': ['mean'],
 'authorized_flag': ['sum', 'mean'],
 'weekend': ['sum', 'mean'],
 'card_id': ['size'],
 'category_1': ['sum', 'mean'],
 'category_2_mean': ['mean'],
 'category_3_mean': ['mean']}

In [49]:
# historical transaction feature engineering
#
# append a column to historical transactions first
for col in ['category_2', 'category_3']:
     historical_transactions[col + '_mean'] = \
        historical_transactions.groupby([col])['purchase_amount'].transform('mean')

# historical transaction features
hist_columns = create_column('hist', aggs)

hist_trans_agg = historical_transactions.groupby('card_id').agg(aggs)
hist_trans_agg.columns = hist_columns
hist_trans_agg.reset_index(drop=False, inplace=False)
hist_trans_agg['hist_purchase_date_diff'] = \
    (hist_trans_agg['hist_purchase_date_max'] - hist_trans_agg['hist_purchase_date_min']).dt.days
hist_trans_agg['hist_purchase_date_average'] = \
    (hist_trans_agg['hist_purchase_date_diff']) / hist_trans_agg['hist_card_id_size']
hist_trans_agg['hist_purchase_date_uptonow'] = \
    (datetime.datetime.today() - hist_trans_agg['hist_purchase_date_max']).dt.days

# merge into train and test dataset
train = train.merge(hist_trans_agg, on='card_id', how='left')
test = test.merge(hist_trans_agg, on='card_id', how='left')

# gc collection, save memory
del hist_trans_agg;gc.collect()

169

In [51]:
# new transaction feature engineering
#
# append a column to new transactions first
for col in ['category_2', 'category_3']:
     new_transactions[col + '_mean'] = \
        new_transactions.groupby([col])['purchase_amount'].transform('mean')

# new transaction features
new_columns = create_column('new', aggs)

new_trans_agg = new_transactions.groupby('card_id').agg(aggs)
new_trans_agg.columns = new_columns
new_trans_agg.reset_index(drop=False, inplace=False)
new_trans_agg['new_purchase_date_diff'] = \
    (new_trans_agg['new_purchase_date_max'] - new_trans_agg['new_purchase_date_min']).dt.days
new_trans_agg['new_purchase_date_average'] = \
    (new_trans_agg['new_purchase_date_diff']) / new_trans_agg['new_card_id_size']
new_trans_agg['new_purchase_date_uptonow'] = \
    (datetime.datetime.today() - new_trans_agg['new_purchase_date_max']).dt.days

# merge into train and test dataset
train = train.merge(new_trans_agg, on='card_id', how='left')
test = test.merge(new_trans_agg, on='card_id', how='left')

# gc collection
del new_trans_agg;gc.collect()

155

In [53]:
# gc collection
del historical_transactions;gc.collect()
del new_transactions;gc.collect()

0

In [54]:
# outliers
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [55]:
# train, test dataset feature engineering
for df in [train, test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['new_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_purchase_date_max',\
                     'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['hist_card_id_size'] + df['new_card_id_size']
    df['purchase_amount_total'] = df['new_purchase_amount_sum'] + df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([f])['outliers'].mean()
    train[f] = train[f].map(order_label)
    test[f] = test[f].map(order_label)

In [56]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,new_purchase_date_uptonow,outliers,dayofweek,weekofyear,month,elapsed_time,hist_first_buy,new_first_buy,card_id_total,purchase_amount_total
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820283,9,23,35,7,...,343.0,0,3,22,6,675,26,277.0,283.0,-179.212942
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392913,12,24,50,7,...,373.0,0,6,52,1,826,5,396.0,356.0,-214.362071
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.688056,10,14,22,7,...,343.0,0,0,31,8,979,163,635.0,44.0,-29.867717
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142495,6,16,20,7,...,354.0,0,4,35,9,583,25,187.0,84.0,-54.145736
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159749,4,22,17,7,...,343.0,0,2,44,11,522,11,121.0,169.0,-68.613893


We can do the modeling now!

In [ ]:
# Use LightGBM to evaluate features